In [1]:
import re
from collections import defaultdict

import os
import matplotlib.pyplot as plt

In [ ]:
def parse_latency_data(filepath):
    # Mapping: (algo, n, ppn) → { size → latency }
    raw_data = defaultdict(dict)
    message_sizes = set()

    with open(filepath, 'r') as f:
        lines = f.readlines()

    current_key = None
    reading_data = False

    for line in lines:
        line = line.strip()

        match = re.match(r"running (\S+) w/ n=(\d+) and ppn=(\d+)", line)
        if match:
            algo, n, ppn = match.groups()
            current_key = (algo, int(n), int(ppn))
            reading_data = False
            continue

        if line.startswith("# Size"):
            reading_data = True
            continue

        if reading_data and line:
            parts = line.split()
            if len(parts) == 2:
                try:
                    size = int(parts[0])
                    latency = float(parts[1])
                    raw_data[current_key][size] = latency
                    message_sizes.add(size)
                except ValueError:
                    continue

    return sorted(message_sizes), raw_data

def group_by_n_ppn(raw_data):
    # Mapping: (n, ppn) → { algo → { size → latency } }
    grouped = defaultdict(lambda: defaultdict(dict))
    for (algo, n, ppn), size_to_latency in raw_data.items():
        grouped[(n, ppn)][algo] = size_to_latency
    return grouped

def plot_data(sizes, data, title="perf", outdir="plots"):
    true_out = f'{outdir}/{title}'
    os.makedirs(true_out, exist_ok=True)

    grouped_data = group_by_n_ppn(data)

    for (n, ppn), algo_dict in grouped_data.items():
        plt.figure(figsize=(10, 6))

        for algo, size_to_latency in algo_dict.items():
            x = [size for size in sizes if size in size_to_latency]
            y = [size_to_latency[size] for size in x]
            plt.plot(x, y, marker='o', label=algo)

        plt.title(f"{title} Latency (n={n}, ppn={ppn})")
        plt.xlabel("Message Size (bytes)")
        plt.ylabel("Avg Latency (us)")
        plt.xscale("log")
        plt.yscale("log")
        plt.grid(True, which="both", ls="--", lw=0.5)
        plt.legend()
        plt.tight_layout()

        filename = f"n{n}_ppn{ppn}.png"
        plt.savefig(os.path.join(true_out, filename))
        plt.close()

In [ ]:
filepath = "ar.out"
sizes, ar = parse_latency_data(filepath)
plot_data(sizes, ar, title="AllRed")

In [ ]:
filepath = "bc.out"
sizes, bc = parse_latency_data(filepath)
plot_data(sizes, bc, title="BCast")